In [1]:
#라이브러리
import tempfile
import os
import zipfile
import tensorflow as tf
import tensorflow_model_optimization as tfmot
import numpy as np
from tensorflow import keras

In [2]:
# 함수 
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

In [3]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=4,
  validation_split=0.1,
)

Epoch 1/4
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2967 - accuracy: 0.9168 - val_loss: 0.1202 - val_accuracy: 0.9688
Epoch 2/4
1688/1688 [==============================] - 6s 3ms/step - loss: 0.1147 - accuracy: 0.9673 - val_loss: 0.0796 - val_accuracy: 0.9797
Epoch 3/4
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0835 - accuracy: 0.9762 - val_loss: 0.0692 - val_accuracy: 0.9817
Epoch 4/4
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0690 - accuracy: 0.9802 - val_loss: 0.0634 - val_accuracy: 0.9835


In [4]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)
print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Baseline test accuracy: 0.9793999791145325
Saved baseline model to: C:\Users\Jay\AppData\Local\Temp\tmpvub9o52r.h5


C:\Users\Jay\AppData\Local\Temp\ipykernel_35088\3664931980.py:6: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model, keras_file, include_optimizer=False)


In [5]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model_for_pruning.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_reshap  (None, 28, 28, 1)         1         
 e (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_conv2d  (None, 26, 26, 12)        230       
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_max_po  (None, 13, 13, 12)        1         
 oling2d (PruneLowMagnitude                                      
 )                                                               
                                                                 
 prune_low_magnitude_flatte  (None, 2028)              1         
 n (PruneLowMagnitude)                                           
                                                        

In [6]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/2
422/422 [==============================] - 5s 9ms/step - loss: 0.0904 - accuracy: 0.9754 - val_loss: 0.1067 - val_accuracy: 0.9730
Epoch 2/2
422/422 [==============================] - 4s 8ms/step - loss: 0.1029 - accuracy: 0.9725 - val_loss: 0.0862 - val_accuracy: 0.9775


In [7]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.9793999791145325
Pruned test accuracy: 0.9750000238418579


In [24]:
print('Baseline test accuracy:', baseline_model_accuracy) 

Baseline test accuracy: 0.9750000238418579


In [25]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

# 프루닝 모델 저장
_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

C:\Users\Jay\AppData\Local\Temp\ipykernel_35088\3008039924.py:5: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)


Saved pruned Keras model to: C:\Users\Jay\AppData\Local\Temp\tmp447v0afc.h5


In [9]:
quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLa  (None, 28, 28)            3         
 yer)                                                            
                                                                 
 quant_reshape (QuantizeWra  (None, 28, 28, 1)         1         
 pperV2)                                                         
                                                                 
 quant_conv2d (QuantizeWrap  (None, 26, 26, 12)        147       
 perV2)                                                          
                                                                 
 quant_max_pooling2d (Quant  (None, 13, 13, 12)        1         
 izeWrapperV2)                                                   
                                                                 
 quant_flatten (QuantizeWra  (None, 2028)              1

In [10]:
train_images_subset = train_images[0:1000] # out of 60000
train_labels_subset = train_labels[0:1000]

q_aware_model.fit(train_images_subset, train_labels_subset,
                  batch_size=500, epochs=1, validation_split=0.1)

2/2 [==============================] - 1s 198ms/step - loss: 0.0973 - accuracy: 0.9789 - val_loss: 0.1211 - val_accuracy: 0.9700


In [11]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

_, q_aware_model_accuracy = q_aware_model.evaluate(
   test_images, test_labels, verbose=0)

In [12]:
#양자화 후 tf 라이트파일로 변환
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\Jay\AppData\Local\Temp\tmpbmnptlvv\assets


INFO:tensorflow:Assets written to: C:\Users\Jay\AppData\Local\Temp\tmpbmnptlvv\assets
c:\Users\Jay\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\lite\python\convert.py:947: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [13]:
interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Quant TFLite test_accuracy:', test_accuracy)
print('Quant TF test accuracy:', q_aware_model_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.


Quant TFLite test_accuracy: 0.9747
Quant TF test accuracy: 0.9746999740600586


In [14]:
# Create float TFLite model.
float_converter = tf.lite.TFLiteConverter.from_keras_model(model)
float_tflite_model = float_converter.convert()

# Measure sizes of models.
_, float_file = tempfile.mkstemp('.tflite')
_, quant_file = tempfile.mkstemp('.tflite')

with open(quant_file, 'wb') as f:
  f.write(quantized_tflite_model)

with open(float_file, 'wb') as f:
  f.write(float_tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Jay\AppData\Local\Temp\tmpvctcxc81\assets


INFO:tensorflow:Assets written to: C:\Users\Jay\AppData\Local\Temp\tmpvctcxc81\assets


In [15]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped Quantized model: %.2f bytes" % (get_gzipped_model_size(quant_file)))

Size of gzipped baseline Keras model: 78312.00 bytes
Size of gzipped pruned Keras model: 25842.00 bytes
Size of gzipped Quantized model: 8585.00 bytes


In [16]:
print("가지치기 압축률: %.2f bytes" % (get_gzipped_model_size(int(keras_file)/int(pruned_keras_file)*100)))

ValueError: invalid literal for int() with base 10: 'C:\\Users\\Jay\\AppData\\Local\\Temp\\tmpvub9o52r.h5'

In [17]:
# 파일 크기를 정수로 변환하여 출력
keras_file_size = int(get_gzipped_model_size(keras_file))
pruned_keras_file_size = int(get_gzipped_model_size(pruned_keras_file))
quant_file_size = int(get_gzipped_model_size(quant_file))

print("Size of gzipped baseline Keras model: %d bytes" % keras_file_size)
print("Size of gzipped pruned Keras model: %d bytes" % pruned_keras_file_size)
print("Size of gzipped Quantized model: %d bytes" % quant_file_size)


Size of gzipped baseline Keras model: 78312 bytes
Size of gzipped pruned Keras model: 25842 bytes
Size of gzipped Quantized model: 8585 bytes


In [18]:
#압축률 = (압축 전 데이터 크기) / (압축 후 데이터 크기)

pruned_keras_compression_rate = ((keras_file_size / pruned_keras_file_size))

In [19]:
quant_file_compression_rate = (keras_file_size / quant_file_size)

In [20]:
print("compression rate of pruned Keras model: %d/10 = 30퍼센트" % pruned_keras_compression_rate)
print("compression rate of Quantized model: %d/10 90퍼센트" % quant_file_compression_rate)

compression rate of pruned Keras model: 3/10 = 30퍼센트
compression rate of Quantized model: 9/10 90퍼센트


In [21]:
# 백분율 정확도 차이(Percentage Accuracy Difference) = [(전체 정확도 - 정확도 차이) / 전체 정확도] * 100%

In [22]:
# 원본과 가지치기 정확도가 동일하게 나옴니다.

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)
print('Quant TF test accuracy:', q_aware_model_accuracy)

Baseline test accuracy: 0.9750000238418579
Pruned test accuracy: 0.9750000238418579
Quant TF test accuracy: 0.9746999740600586
